In [234]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

In [235]:
#get data from csv
data_jobs = pd.read_csv('glassdoor_jobs.csv', index_col=0)
data_jobs.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),-1
1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1
2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,Company - Private,Security Services,Business Services,$100 to $500 million (USD),-1
3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa..."
4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"


In [236]:
data_jobs.describe()

,Rating,Founded
count,956.000000,956.000000
mean,3.601255,1774.605649
std,1.067619,598.942517
min,-1.000000,-1.000000
25%,3.300000,1937.000000
50%,3.800000,1992.000000
75%,4.200000,2008.000000
max,5.000000,2019.000000


In [237]:
data_jobs[['Salary Estimate']].isnull().values.any()

False

In [238]:
data_jobs = data_jobs[data_jobs['Salary Estimate'] != '-1']

In [239]:
data_jobs['Salary Estimate'] = data_jobs['Salary Estimate'].apply(lambda x: x.lower().replace('k', '').replace('$', '').replace('(glassdoor est.)', '').replace('employer provided salary:', '').replace('(employer est.)', '').replace(' per hour', ''))

In [240]:
data_jobs['Employer Provided Salary'] = data_jobs['Salary Estimate'].apply(lambda x: 1 if 'employer provided salary' in x.lower() else 0)

In [241]:
data_jobs['Hourly Paid'] = data_jobs['Salary Estimate'].apply(lambda x: 1 if 'per hour' in x.lower() else 0)

In [242]:
data_jobs['Min Salary'] = data_jobs['Salary Estimate'].apply(lambda x: int(x.split('-')[0]))

In [243]:
data_jobs['Max Salary'] = data_jobs['Salary Estimate'].apply(lambda x: int(x.split('-')[1]))

In [244]:
data_jobs['Avg Salary'] = (data_jobs['Min Salary']+data_jobs['Max Salary'])/2

In [245]:
#refactoring seniority and job title

def job_title_refactor(title):
    if 'manager' in title.lower():
        return 'manager'
    elif 'director' in title.lower():
        return 'director'
    elif 'data engineer' in title.lower():
        return 'data engineer'
    elif 'scientist' in title.lower():
        return 'data scientist'
    elif 'analyst' in title.lower() or 'analytics' in title.lower():
        return 'data analyst'
    elif 'machine learning' in title.lower():
        return 'mle'
    else:
        return 'na'

def seniority_refactor(seniority):
    if 'senior' in seniority.lower() or 'sr' in seniority.lower() or 'sr.' in seniority.lower() or 'principal' in seniority.lower():
        return 'senior'
    elif 'junior' in seniority.lower() or 'jr' in seniority.lower() or 'jr.' in seniority.lower():
        return 'junior'
    else:
        return 'na'

In [246]:
data_jobs['Simple Job Title'] = data_jobs['Job Title'].apply(job_title_refactor)

In [247]:
data_jobs['Simple Job Title'].value_counts()

data scientist    431
data engineer     116
data analyst      112
na                 29
manager            25
director           17
mle                12
Name: Simple Job Title, dtype: int64

In [248]:
data_jobs['Seniority'] = data_jobs['Job Title'].apply(seniority_refactor)

In [249]:
data_jobs['Seniority'].value_counts()

na        540
senior    199
junior      3
Name: Seniority, dtype: int64

In [250]:
ratings = data_jobs[data_jobs['Rating'] != '-1']
ratings['Company Name'] = ratings['Company Name'].apply(lambda x: x[:-5])
data_jobs[data_jobs['Rating'] != '-1'] = ratings
data_jobs

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,...,Sector,Revenue,Competitors,Employer Provided Salary,Hourly Paid,Min Salary,Max Salary,Avg Salary,Simple Job Title,Seniority
0,Data Scientist,53-91,"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Researc,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,...,Aerospace & Defense,$50 to $100 million (USD),-1,0,0,53,91,72.0,data scientist,na
1,Healthcare Data Scientist,63-112,What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical Syste,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,...,Health Care,$2 to $5 billion (USD),-1,0,0,63,112,87.5,data scientist,na
2,Data Scientist,80-90,"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,Company - Private,...,Business Services,$100 to $500 million (USD),-1,0,0,80,90,85.0,data scientist,na
3,Data Scientist,56-97,*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNN,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,...,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa...",0,0,56,97,76.5,data scientist,na
4,Data Scientist,86-143,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solution,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,...,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee",0,0,86,143,114.5,data scientist,na
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950,"Sr Scientist, Immuno-Oncology - Oncology",58-111,Site Name: USA - Massachusetts - Cambridge\nPo...,3.9,GS,"Cambridge, MA","Brentford, United Kingdom",10000+ employees,1830,Company - Public,...,Biotech & Pharmaceuticals,$10+ billion (USD),"Pfizer, AstraZeneca, Merck",0,0,58,111,84.5,data scientist,senior
951,Senior Data Engineer,72-133,THE CHALLENGE\nEventbrite has a world-class da...,4.4,Eventbrit,"Nashville, TN","San Francisco, CA",1001 to 5000 employees,2006,Company - Public,...,Information Technology,$100 to $500 million (USD),"See Tickets, TicketWeb, Vendini",0,0,72,133,102.5,data engineer,senior
952,"Project Scientist - Auton Lab, Robotics Institute",56-91,The Auton Lab at Carnegie Mellon University is...,2.6,Software Engineering Institut,"Pittsburgh, PA","Pittsburgh, PA",501 to 1000 employees,1984,College / University,...,Education,Unknown / Non-Applicable,-1,0,0,56,91,73.5,data scientist,na
953,Data Science Manager,95-160,Data Science ManagerResponsibilities:\n\nOvers...,3.2,"Numeric, LL","Allentown, PA","Chadds Ford, PA",1 to 50 employees,-1,Company - Private,...,Business Services,$5 to $10 million (USD),-1,0,0,95,160,127.5,manager,na


In [355]:
data_jobs['Competitor count'] = data_jobs['Competitors'].apply(lambda x: len(x.split(',')) if x != '-1' else 0)
data_jobs[['Competitors', 'Competitor count']]

,Competitors,Competitor count
0,-1,0
1,-1,0
2,-1,0
3,"Oak Ridge National Laboratory, National Renewa...",3
4,"Commerce Signals, Cardlytics, Yodlee",3
...,...,...
950,"Pfizer, AstraZeneca, Merck",3
951,"See Tickets, TicketWeb, Vendini",3
952,-1,0
953,-1,0


In [251]:
data_jobs['State'] = data_jobs['Location'].apply(lambda x: x.split(',')[1])
data_jobs['State'].value_counts()

 CA             151
 MA             103
 NY              72
 VA              41
 IL              40
 MD              35
 PA              33
 TX              28
 WA              21
 NC              21
 NJ              17
 FL              16
 OH              14
 TN              13
 DC              11
 CO              11
 WI              10
 IN              10
 UT              10
 MO               9
 AZ               9
 AL               8
 KY               6
 MI               6
 DE               6
 GA               6
 IA               5
 CT               5
 LA               4
 OR               4
 NE               4
 KS               3
 NM               3
 ID               2
 MN               2
 Los Angeles      1
 RI               1
 SC               1
Name: State, dtype: int64

In [252]:
data_jobs['State'] = data_jobs['State'].apply(lambda x: 'CA' if 'los angeles' in x.lower() else x.strip())
data_jobs.State.value_counts()

CA    152
MA    103
NY     72
VA     41
IL     40
MD     35
PA     33
TX     28
NC     21
WA     21
NJ     17
FL     16
OH     14
TN     13
CO     11
DC     11
WI     10
UT     10
IN     10
MO      9
AZ      9
AL      8
GA      6
DE      6
MI      6
KY      6
IA      5
CT      5
NE      4
OR      4
LA      4
NM      3
KS      3
ID      2
MN      2
SC      1
RI      1
Name: State, dtype: int64

In [253]:
current_year = dt.datetime.now().year
data_jobs['Company Age'] = data_jobs['Founded'].apply(lambda x: current_year-x if x != -1 else 'na')

In [343]:
data_jobs['Python'] = data_jobs['Job Description'].apply(lambda x: 1 if 'python' in x.lower() else 0)
data_jobs['Python'].value_counts()
data_jobs['Python'].head(20)

0     1
1     1
2     1
3     1
4     1
5     1
6     0
7     1
8     0
9     1
10    1
11    0
12    0
13    1
14    1
15    0
16    0
17    1
18    0
19    1
Name: Python, dtype: int64

In [341]:
data_jobs['R'] = data_jobs['Job Description'].apply(lambda x: 1 if 'r studio' in x.lower() or 'r-studio' in x.lower() else 0)
data_jobs['R'].value_counts()

0    740
1      2
Name: R, dtype: int64

In [264]:
data_jobs['Spark'] = data_jobs['Job Description'].apply(lambda x: 1 if 'spark' in x.lower() else 0)
data_jobs['Spark'].value_counts()

0    575
1    167
Name: Spark, dtype: int64

In [345]:
data_jobs['Aws'] = data_jobs['Job Description'].apply(lambda x: 1 if 'aws' in x.lower() else 0)
data_jobs.Aws.value_counts()

0    566
1    176
Name: Aws, dtype: int64

In [313]:
data_jobs['BI tools'] = data_jobs['Job Description'].apply(lambda x: 1 if 'tableau' in x.lower()
                                                           or 'power bi' in x.lower()
                                                           or 'sisense' in x.lower()
                                                           or 'microstrategy' in x.lower()
                                                           or 'microsoft power bi' in x.lower()
                                                           or 'looker' in x.lower()
                                                           or 'domo' in x.lower()
                                                           or 'bi tools' in x.lower() else 0)
data_jobs['BI tools'].value_counts()

0    576
1    166
Name: BI tools, dtype: int64

In [344]:
data_jobs['Excel'] = data_jobs['Job Description'].apply(lambda x: 1 if 'excel' in x.lower() else 0)
data_jobs['Excel'].value_counts()

1    388
0    354
Name: Excel, dtype: int64

In [350]:
data_jobs.columns

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors',
       'Employer Provided Salary', 'Hourly Paid', 'Min Salary', 'Max Salary',
       'Avg Salary', 'Simple Job Title', 'Seniority', 'State', 'Company Age',
       'Spark', 'R', 'Power BI', 'BI tools', 'Python', 'Excel', 'Aws'],
      dtype='object')